-limpieza

In [1]:
#importacion de las s librerias que usaremos en este caso.
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import re

import matplotlib.pyplot as plt
from wordcloud import WordCloud


-importacion de los datos.
    -importamos en el datafraeme'aa' referencia a 'accidentes aereos'

In [2]:

aa= pd.read_csv('AccidentesAviones.csv',encoding='utf-8')
#en alguno intentos de importacion sin usar 'utf-8' se mesclan algunas columnas.
aa.head()


,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [3]:
# numero de columnas y filas.
aa.shape


(5008, 18)

In [4]:
#revisemos el archivo para reconocer sus columnas y comprender que informacion nos brinda.
aa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5008 non-null   int64 
 1   fecha                   5008 non-null   object
 2   HORA declarada          5008 non-null   object
 3   Ruta                    5008 non-null   object
 4   OperadOR                5008 non-null   object
 5   flight_no               5008 non-null   object
 6   route                   5007 non-null   object
 7   ac_type                 5008 non-null   object
 8   registration            5008 non-null   object
 9   cn_ln                   5008 non-null   object
 10  all_aboard              5008 non-null   object
 11  PASAJEROS A BORDO       5008 non-null   object
 12  crew_aboard             5008 non-null   object
 13  cantidad de fallecidos  5008 non-null   object
 14  passenger_fatalities    5008 non-null   object
 15  crew

-Dicionario de datos en tabla:
    -unnamed - index antiguo de la lista

    -fecha - fecha del accidente.

    -hora_declarada- hora local formato 24h hh:mm.

    -ruta - locacion del accidente.

    -operador - aerolinea en operacion de la aeronave.

    -numero de vuelo - nuemero de vuelo asignado a esta aeronave.

    -ruta_vuelo- ruta que cumplia la aeronave.

    -tipo - tipo de aeronave.

    -icao registro - registro icao de la aeronave.

    -cn_ln -serie de la aeronave .

    -all board- total personas en el vuelo.

    -pasajeros a bordo- pasajeros a bordo.

    -crew aboard -tripulantes a bordo.

    -total fatalities -total de vidas perdidas.

    -passenger fatalities - pasajeros fallecidos.

    -crew fatalities - tripulantes fallecidos.

    -ground - total de muertos en tierra

    -summary - breve descripcion del accidente y s.u posible causa.

todas las columnas estan en dtype object. lo que quiere decir que son texto o son interpretadas asi.
    

    

In [5]:
# primero asignarle un nombre entendible a las columnas.
aa.rename(columns={ 'fecha': 'fecha', 
                    'HORA declarada': 'hora',
                    'Ruta': 'pais',
                    'OperadOR': 'operadora',
                    'flight_no': 'vuelo_numero',
                    'route' : 'ruta',
                    'ac_type': 'tipo_aeronave',
                    'registration': 'registro_aeronave',
                    'cn_ln': 'registro_fuselaje',
                    'all_aboard': 'abordo',
                    'PASAJEROS A BORDO': 'pasajeros',
                    'crew_aboard': 'tripulantes',
                    'cantidad de fallecidos': 'bajas',
                    'passenger_fatalities': 'bajas_pasajeros',
                    'crew_fatalities': 'bajas_tripulacion',
                    'ground': 'bajas_tierra',
                    'summary': 'Descripcion'
}, inplace=True)
aa

,Unnamed: 0,fecha,hora,pais,operadora,vuelo_numero,ruta,tipo_aeronave,registro_aeronave,registro_fuselaje,abordo,pasajeros,tripulantes,bajas,bajas_pasajeros,bajas_tripulacion,bajas_tierra,Descripcion
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,"March 28, 2021",1835,"Near Butte, Alaska",Soloy Helicopters,?,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...
5004,5004,"May 21, 2021",1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,?,?,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi..."
5005,5005,"June 10, 2021",0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,?,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...
5006,5006,"July 04, 2021",11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,?,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,88,8,50,?,?,3,"While attempting to land at Jolo Airport, the ..."


- una vez definido nuestro diccionario hay que darle un formato a cada columna.

In [6]:
# visualizamos el tipo de datos para formatear.
aa.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         5008 non-null   int64 
 1   fecha              5008 non-null   object
 2   hora               5008 non-null   object
 3   pais               5008 non-null   object
 4   operadora          5008 non-null   object
 5   vuelo_numero       5008 non-null   object
 6   ruta               5007 non-null   object
 7   tipo_aeronave      5008 non-null   object
 8   registro_aeronave  5008 non-null   object
 9   registro_fuselaje  5008 non-null   object
 10  abordo             5008 non-null   object
 11  pasajeros          5008 non-null   object
 12  tripulantes        5008 non-null   object
 13  bajas              5008 non-null   object
 14  bajas_pasajeros    5008 non-null   object
 15  bajas_tripulacion  5008 non-null   object
 16  bajas_tierra       5008 non-null   object


In [7]:
# retiro las columnas 'Unnamed:' y el registro de fuselaje en este caso no los usare.
aa.drop(columns=['Unnamed: 0','registro_fuselaje'], inplace = True)
aa.columns

Index(['fecha', 'hora', 'pais', 'operadora', 'vuelo_numero', 'ruta',
       'tipo_aeronave', 'registro_aeronave', 'abordo', 'pasajeros',
       'tripulantes', 'bajas', 'bajas_pasajeros', 'bajas_tripulacion',
       'bajas_tierra', 'Descripcion'],
      dtype='object')

In [8]:
# seguimos con las mismas filas pero retiramos 2 columnas.
aa.shape

(5008, 16)

Empecemos...


In [9]:
aa['fecha']

0       September 17, 1908
1       September 07, 1909
2            July 12, 1912
3          August 06, 1913
4       September 09, 1913
               ...        
5003        March 28, 2021
5004          May 21, 2021
5005         June 10, 2021
5006         July 04, 2021
5007         July 06, 2021
Name: fecha, Length: 5008, dtype: object

In [10]:
#crear una funcion que formatee la columna fecha a dtype 'datatime64' 
#permitiendo asi usarla adecuadamente.
def convertir_fecha(texto_fecha, formato_entrada, formato_salida):
    try:
        fecha_objeto = datetime.strptime(texto_fecha, formato_entrada)
        fecha_formateada = fecha_objeto.strftime(formato_salida)
        return fecha_formateada
    except ValueError:
        return None



# Especificar los formatos de entrada y salida
formato_entrada = '%B %d, %Y'
formato_salida = '%Y-%m-%d'

# Aplicar la función de conversión a la columna 'fecha'
aa['fecha'] = aa['fecha'].apply(lambda x: convertir_fecha(x, formato_entrada, formato_salida))

# Reemplazar valores nulos por ''
aa['fecha'] = aa['fecha'].fillna('')

# Reemplazar valores '?' por un valor vacío
aa['fecha'] = aa['fecha'].str.replace('?', '')

# Convertir la columna 'fecha' al formato datetime
aa['fecha'] = pd.to_datetime(aa['fecha'], format=formato_salida)

aa['fecha'].head()

0   1908-09-17
1   1909-09-07
2   1912-07-12
3   1913-08-06
4   1913-09-09
Name: fecha, dtype: datetime64[ns]

-----------------------------------------------------------------------------

In [11]:
aa.info() # ya tenemos nuestra primer columna lista.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha              5008 non-null   datetime64[ns]
 1   hora               5008 non-null   object        
 2   pais               5008 non-null   object        
 3   operadora          5008 non-null   object        
 4   vuelo_numero       5008 non-null   object        
 5   ruta               5007 non-null   object        
 6   tipo_aeronave      5008 non-null   object        
 7   registro_aeronave  5008 non-null   object        
 8   abordo             5008 non-null   object        
 9   pasajeros          5008 non-null   object        
 10  tripulantes        5008 non-null   object        
 11  bajas              5008 non-null   object        
 12  bajas_pasajeros    5008 non-null   object        
 13  bajas_tripulacion  5008 non-null   object        
 14  bajas_ti

In [12]:
# formatizamos la columna 'hora'
# comencemos poniendo ':' entre los numeros 
aa['hora'] = aa['hora'].str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)

In [13]:
aa.shape

(5008, 16)

In [14]:
# filtramos nulos, reemplazandolo por una '' vacio.
aa['hora'] = aa['hora'].fillna('')

# convierto la hora en texto
aa['hora'] = aa['hora'].astype(str)

# Elimina los caracteres '?' de la columna
aa['hora'] = aa['hora'].str.replace('?', '')

# Elimina espacios en blanco alrededor de los valores
aa['hora'] = aa['hora'].str.strip()

# Extraemos los valores correspondientes a 'HH:MM'  
aa['hora'] = aa['hora'].str.extract(r'(\d{2}:\d{2})', expand=False)

# Rellena los valores nulos en 'hora' con una cadena vacía
aa['hora'].fillna('', inplace=True)

# Se le da formato a la hora con datetime, muestra solo la hora y convierte en string.
aa['hora'] = pd.to_datetime(aa['hora'], format='%H:%M', errors='coerce')
aa['hora'] = aa['hora'].dt.strftime('%H:%M')
aa['hora'] = aa['hora'].astype(str)


In [15]:
aa['hora']
# listo ya tenemos las dos primeras columnas.


0       17:18
1         nan
2       06:30
3         nan
4       18:30
        ...  
5003    18:35
5004    18:00
5005    08:00
5006    11:30
5007    15:00
Name: hora, Length: 5008, dtype: object

In [16]:
# la columna 'pais' dejare solo el pais donde sucedio el accidente segun la documentacion.
aa.pais.unique()

array(['Fort Myer, Virginia', 'Juvisy-sur-Orge, France',
       'Atlantic City, New Jersey', ..., 'Near Pyin Oo Lwin, Myanmar',
       'Patikul, Sulu, Philippines', 'Palana, Russia'], dtype=object)

In [17]:
aa.pais.nunique()

4125

In [18]:
aa['pais'] = aa['pais'].fillna('')
aa['pais'] = aa['pais'].str.replace('?', '')

In [19]:
# una lista con los países
lista_paises = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Côte d'Ivoire", "Cabo Verde", "Cambodia", "Cameroon", "Canada",
    "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Croatia",
    "Cuba", "Cyprus", "Czech Republic", "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica",
    "Dominican Republic", "Ecuador", "England", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia",
    "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece",
    "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary",
    "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy","Ivory Coast", "Jamaica", "Japan", "Jordan",
    "Kazakhstan", "Kenya", "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia",
    "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali",
    "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia",
    "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand",
    "Nicaragua", "Nigeria", "Niger", "North Korea", "North Macedonia", "Norway", "Oman", "Pakistan", "Palau",
    "Palestine State", "Panama", "Papua", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Puerto Rico",
    "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Scotland",
    "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia",
    "South Africa", "South Korea", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland",
    "Syria", "Tajikistan", "Taiwan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago",
    "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom",
    "United States", "Uruguay", "USSR", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

#definimos una función para mostrar solo el país en la columna
def obtener_pais(location):
    for pais in lista_paises:
        if pais.lower() in location.lower():
            return pais
    return 'other'

# Aplicamos la función 'obtener_país' a la columna 'pais' y creamos la columna 'pais'
aa['pais'] = aa['pais'].apply(obtener_pais)

In [21]:
#vamos con la columna 'operadora'
# Reemplazamos los valores nulos por un valor vacío
aa['operadora'] = aa['operadora'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['operadora'] = aa['operadora'].str.replace('?', '')


In [22]:
# Reemplazamos los valores nulos por un valor vacío
aa['vuelo_numero'] = aa['vuelo_numero'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['vuelo_numero'] = aa['vuelo_numero'].str.replace('?', '')


In [23]:


# Reemplazamos los valores nulos por un valor vacío
aa['ruta'] = aa['ruta'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['ruta'] = aa['ruta'].str.replace('?', '')



In [24]:
# Reemplazamos los valores nulos por un valor vacío
aa['tipo_aeronave'] = aa['tipo_aeronave'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['tipo_aeronave'] = aa['tipo_aeronave'].str.replace('?', '')

In [25]:
#tomamos  una lista de marcas de aeronaves 
marcas_aeronaves_historicas = ['Wright Brothers', 'Boeing', 'Airbus', 'Lockheed Martin', 'McDonnell Douglas',
                              'Northrop Grumman', 'Convair', 'Grumman', 'Vickers', 'De Havilland', 'Hawker',
                              'Avro', 'Fokker', 'Junkers', 'Douglas Aircraft Company', 'Cessna', 'Piper Aircraft',
                              'Beechcraft', 'Bell Aircraft Corporation', 'Martin Marietta', 'Fairchild Aircraft',
                              'Bristol Aeroplane Company', 'Handley Page', 'Curtiss-Wright Corporation', 'Saab',
                              'Aérospatiale', 'Tupolev', 'Ilyushin', 'Antonov', 'Yakovlev', 'Sud Aviation',
                              'Dornier', 'Breguet Aviation', 'Messerschmitt', 'Vought', 'Piaggio Aerospace',
                              'Sikorsky Aircraft Corporation', 'Short Brothers', 'Supermarine', 'Avro Canada',
                              'Harbin Aircraft Manufacturing Corporation', 'English Electric',
                              'Mitsubishi Heavy Industries', 'Aichi', 'Nakajima', 'Kawasaki Heavy Industries',
                              'Handley Page', 'Armstrong Whitworth Aircraft', 'Blackburn Aircraft',
                              'Sopwith Aviation Company']

In [26]:
# Función para asignar valores a la columna 'brand'
def assign_brand(aircraft_type):
    for brand in marcas_aeronaves_historicas:
        if any(word.lower() in aircraft_type.lower().split() for word in brand.split()):
            return brand
    return 'other'

# Aplicamos la función a la columna 'aircraft_type' para crear la columna 'brand'
aa['tipo_aeronave'] = aa['tipo_aeronave'].apply(assign_brand)

In [27]:
# Reemplazar los valores nulos por un valor vacío
aa['abordo'] = aa['abordo'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['abordo'] = aa['abordo'].str.replace('?', '')

# Eliminar filas con valores que no son números válidos (NaN y cadenas no numéricas)
aa = aa[pd.to_numeric(aa['abordo'], errors='coerce').notna()]

# Convertir la columna 'a bordo' a tipo entero (int)
aa['abordo'] = aa['abordo'].astype(int)


/tmp/ipykernel_39968/81569395.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['abordo'] = aa['abordo'].astype(int)


In [28]:
# Reemplazamos los valores nulos por un valor vacío
aa['registro_aeronave'] = aa['registro_aeronave'] .fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['registro_aeronave']  = aa['registro_aeronave'] .str.replace('?', '')

# Convertimos la columna 'a bordo' a tipo numérico 
aa['registro_aeronave']  = pd.to_numeric(aa['registro_aeronave'] , errors='coerce')

/tmp/ipykernel_39968/3674777899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['registro_aeronave'] = aa['registro_aeronave'] .fillna('')
/tmp/ipykernel_39968/3674777899.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['registro_aeronave']  = aa['registro_aeronave'] .str.replace('?', '')
/tmp/ipykernel_39968/3674777899.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [29]:
# Reemplazamos los valores nulos por un valor vacío
aa['bajas'] = aa['bajas'].fillna('')

# Reemplazamos los valores de ? por un valor vacío
aa['bajas'] = aa['bajas'].str.replace('?', '')

# Convertimos la columna 'total_abord' a tipo numérico 
aa['bajas'] = pd.to_numeric(aa['bajas'], errors='coerce')

/tmp/ipykernel_39968/2989187705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas'] = aa['bajas'].fillna('')
/tmp/ipykernel_39968/2989187705.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas'] = aa['bajas'].str.replace('?', '')
/tmp/ipykernel_39968/2989187705.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [30]:
# Reemplazar los valores nulos por un valor vacío
aa['tripulantes'] = aa['tripulantes'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['tripulantes'] = aa['tripulantes'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['tripulantes'] = pd.to_numeric(aa['tripulantes'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['tripulantes'] = aa['tripulantes'].fillna(pd.NA).astype('Int64')


/tmp/ipykernel_39968/868821141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['tripulantes'] = aa['tripulantes'].fillna('')
/tmp/ipykernel_39968/868821141.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['tripulantes'] = aa['tripulantes'].str.replace('?', '')
/tmp/ipykernel_39968/868821141.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [31]:
# Reemplazar los valores nulos por un valor vacío
aa['bajas_pasajeros'] = aa['bajas_pasajeros'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['bajas_pasajeros'] = aa['bajas_pasajeros'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['bajas_pasajeros'] = pd.to_numeric(aa['bajas_pasajeros'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['bajas_pasajeros'] = aa['bajas_pasajeros'].fillna(pd.NA).astype('Int64')


/tmp/ipykernel_39968/2228817691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_pasajeros'] = aa['bajas_pasajeros'].fillna('')
/tmp/ipykernel_39968/2228817691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_pasajeros'] = aa['bajas_pasajeros'].str.replace('?', '')
/tmp/ipykernel_39968/2228817691.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [ ]:
# División política de Afganistán
divisiones_afganistan = [
    "Badakhshan", "Badghis", "Baghlan", "Balkh", "Bamyan", "Daykundi", "Farah", "Faryab", "Ghazni", "Ghor", "Helmand",
    "Herat", "Jowzjan", "Kabul", "Kandahar", "Kapisa", "Khost", "Kunar", "Kunduz", "Laghman", "Logar", "Nangarhar",
    "Nimruz", "Nuristan", "Paktia", "Paktika", "Panjshir", "Parwan", "Samangan", "Sar-e Pol", "Takhar", "Urozgan",
    "Zabul"
]

# División política de Albania
divisiones_albania = [
    "Condado de Berat", "Condado de Dibër", "Condado de Durrës", "Condado de Elbasan", "Condado de Fier", "Condado de Gjirokastër",
    "Condado de Korçë", "Condado de Kukës", "Condado de Lezhë", "Condado de Shkodër", "Condado de Tiranë", "Condado de Vlorë"
]

# División política de Argelia
divisiones_argelia = [
    "Adrar", "Chlef", "Laghouat", "Oum El Bouaghi", "Batna", "Béjaïa", "Biskra", "Béchar", "Blida", "Bouira", "Tamanrasset",
    "Tébessa", "Tlemcen", "Tiaret", "Tizi Ouzou", "Alger", "Djelfa", "Jijel", "Sétif", "Saïda", "Skikda", "Sidi Bel Abbès",
    "Annaba", "Guelma", "Constantine", "Médéa", "Mostaganem", "M'Sila", "Mascara", "Ouargla", "Oran", "El Bayadh", "Illizi",
    "Bordj Bou Arréridj", "Boumerdès", "El Tarf", "Tindouf", "Tissemsilt", "El Oued", "Khenchela", "Souk Ahras", "Tipaza",
    "Mila", "Aïn Defla", "Naâma", "Aïn Témouchent", "Ghardaïa", "Relizane"
]
# División política de Andorra
divisiones_andorra = ["Andorra la Vella", "Canillo", "Encamp", "Escaldes-Engordany", "La Massana", "Ordino", "Sant Julià de Lòria"]

# División política de Angola
divisiones_angola = [
    "Bengo", "Benguela", "Bié", "Cabinda", "Cuando Cubango", "Cuanza Norte", "Cuanza Sul", "Cunene", "Huambo", "Huíla",
    "Luanda", "Lunda Norte", "Lunda Sul", "Malanje", "Moxico", "Namibe", "Uíge", "Zaire"
]

# División política de Antigua y Barbuda
divisiones_antigua_barbuda = ["Antigua", "Barbuda", "Redonda"]

# División política de Argentina
divisiones_argentina = [
    "Buenos Aires", "Catamarca", "Chaco", "Chubut", "Córdoba", "Corrientes", "Entre Ríos", "Formosa", "Jujuy",
    "La Pampa", "La Rioja", "Mendoza", "Misiones", "Neuquén", "Río Negro", "Salta", "San Juan", "San Luis", "Santa Cruz",
    "Santa Fe", "Santiago del Estero", "Tierra del Fuego", "Tucumán", "Ciudad Autónoma de Buenos Aires"
]

# División política de Armenia
divisiones_armenia = [
    "Aragatsotn", "Ararat", "Armavir", "Gegharkunik", "Kotayk", "Lori", "Shirak", "Syunik", "Tavush", "Vayots Dzor", "Yerevan"
]

# División política de Australia
divisiones_australia = [
    "Nueva Gales del Sur", "Queensland", "Australia Meridional", "Tasmania", "Victoria", "Australia Occidental",
    "Territorio de la Capital Australiana", "Territorio del Norte", "Isla de Norfolk", "Islas Heard y McDonald",
    "Islas Cocos (Keeling)", "Islas Ashmore y Cartier", "Territorio Antártico Australiano"
]

# División política de Austria
divisiones_austria = [
    "Burgenland", "Carintia", "Baja Austria (Niederösterreich)", "Alta Austria (Oberösterreich)", "Salzburgo (Salzburg)",
    "Estiria (Steiermark)", "Tirol (Tyrol)", "Vorarlberg", "Viena (Wien)"
]

# División política de Azerbaiyán
divisiones_azerbaiyan = [
    "Agsu", "Astara", "Baku", "Balakan", "Barda", "Beylagan", "Bilasuvar", "Dashkasan", "Fizuli", "Ganja", "Goranboy",
    "Goychay", "Goygol", "Hajigabul", "Imishli", "Ismayilli", "Jabrayil", "Julfa", "Kalbajar", "Kangarli", "Khachmaz",
    "Khizi", "Khojaly", "Khudat", "Lachin", "Lankaran", "Lankaran", "Lerik", "Martuni", "Masally", "Mingachevir", "Naftalan",
    "Nakhchivan", "Neftchala", "Oghuz", "Qabala", "Qakh", "Qazakh", "Quba", "Qubadli", "Qusar", "Saatly", "Sabirabad",
    "Salyan", "Shabran", "Shaki", "Shamakhi", "Shamkir", "Shirvan", "Siazan", "Sumqayit", "Tartar", "Tovuz", "Ujar", "Yardimli",
    "Yevlakh", "Zangilan", "Zaqatala", "Zardab"
]

# División política de Bahamas
divisiones_bahamas = [
    "Acklins", "Berry Islands", "Bimini", "Black Point", "Cat Island", "Central Abaco", "Central Andros",
    "Central Eleuthera", "City of Freeport", "Crooked Island and Long Cay", "East Grand Bahama", "Exuma",
    "Grand Cay", "Harbour Island", "Hope Town", "Inagua", "Long Island", "Mangrove Cay", "Mayaguana",
    "Moore's Island", "North Abaco", "North Andros", "North Eleuthera", "Ragged Island", "Rum Cay", "San Salvador",
    "South Abaco", "South Andros", "South Eleuthera", "Spanish Wells", "West Grand Bahama"
]

# División política de Bahréin
divisiones_bahrein = ["Capital", "Muharraq", "Norte", "Sur"]

# División política de Bangladesh
divisiones_bangladesh = [
    "Bagerhat", "Bandarban", "Barguna", "Barisal", "Bhola", "Bogra", "Brahmanbaria", "Chandpur", "Chapai Nawabganj",
    "Chittagong", "Chuadanga", "Comilla", "Cox's Bazar", "Dhaka", "Dinajpur", "Faridpur", "Feni", "Gaibandha",
    "Gazipur", "Gopalganj", "Habiganj", "Jamalpur", "Jashore", "Jhalokati", "Jhenaidah", "Joypurhat", "Khagrachari",
    "Khulna", "Kishoreganj", "Kurigram", "Kushtia", "Lakshmipur", "Lalmonirhat", "Madaripur", "Magura", "Manikganj",
    "Meherpur", "Moulvibazar", "Munshiganj", "Mymensingh", "Naogaon", "Narail", "Narayanganj", "Narsingdi", "Natore",
    "Nawabganj", "Netrokona", "Nilphamari", "Noakhali", "Pabna", "Panchagarh", "Patuakhali", "Pirojpur", "Rajbari",
    "Rajshahi", "Rangamati", "Rangpur", "Satkhira", "Shariatpur", "Sherpur", "Sirajganj", "Sunamganj", "Sylhet",
    "Tangail", "Thakurgaon"
]

# División política de Barbados
divisiones_barbados = ["Christ Church", "Saint Andrew", "Saint George", "Saint James", "Saint John", "Saint Joseph", "Saint Lucy", "Saint Michael", "Saint Peter", "Saint Philip", "Saint Thomas"]

# División política de Belarús (Bielorrusia)
divisiones_biolorusia = [
    "Brest", "Gómel", "Grodno", "Minsk", "Mogilev", "Vítebsk", "Brest", "Brest", "Brest", "Brest", "Brest", "Brest", "Brest"
]

# División política de Bélgica
divisiones_belgica = [
    "Amberes", "Brabante Flamenco", "Brabante Valón", "Bruselas-Capital", "Flandes Occidental", "Flandes Oriental", "Hainaut",
    "Lieja", "Limburgo", "Luxemburgo", "Namur"
]

# División política de Belice
divisiones_belice = [
    "Belice", "Cayo", "Corozal", "Orange Walk", "Stann Creek", "Toledo"
]

# División política de Benín
divisiones_benin = [
    "Alibori", "Atacora", "Atlántico", "Borgou", "Collines", "Donga", "Kouffo", "Littoral", "Mono", "Ouémé", "Plateau", "Zou"
]

# División política de Bután
divisiones_bután = [
    "Bumthang", "Chhukha", "Dagana", "Gasa", "Haa", "Lhuntse", "Mongar", "Paro", "Pemagatshel", "Punakha", "Samdrup Jongkhar", "Samtse", "Sarpang", "Thimphu", "Trashigang", "Trongsa", "Tsirang", "Wangdue Phodrang", "Zhemgang"
]

# División política de Bolivia
divisiones_bolivia = [
    "Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosí", "Santa Cruz", "Tarija"
]

# División política de Bosnia y Herzegovina
divisiones_bosnia_herzegovina = [
    "Federación de Bosnia y Herzegovina", "Republika Srpska", "Brcko"
]

# División política de Botsuana
divisiones_botsuana = [
    "Central", "Ghanzi", "Kgalagadi", "Kgatleng", "Kweneng", "Ngamiland", "Northeast", "Northwest", "Southern"
]

# División política de Brasil
divisiones_brasil = [
    "Acre", "Alagoas", "Amapá", "Amazonas", "Bahía", "Ceará", "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão",
    "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco", "Piauí", "Rio de Janeiro",
    "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima", "Santa Catarina", "São Paulo", "Sergipe", "Tocantins"
]

# División política de Brunei
divisiones_brunei = [
    "Belait", "Brunei-Muara", "Temburong", "Tutong"
]

# División política de Bulgaria
divisiones_bulgaria = [
    "Blagoevgrad", "Burgas", "Dobrich", "Gabrovo", "Haskovo", "Kardzhali", "Kyustendil", "Lovech", "Montana", "Pazardzhik", "Pernik", "Pleven", "Plovdiv", "Razgrad", "Ruse", "Shumen", "Silistra", "Sliven", "Smolyan", "Sofia", "Sofia-Grad", "Stara Zagora", "Targovishte", "Varna", "Veliko Tarnovo", "Vidin", "Vratsa", "Yambol"
]
# División política de Burkina Faso
divisiones_burkina_faso = [
    "Balé", "Bam", "Banwa", "Bazèga", "Bougouriba", "Boulgou", "Boulkiemdé", "Comoé", "Déwé", "Ganzourgou", "Gnagna", "Gourma", "Houet", "Ioba", "Kadiogo", "Kénédougou", "Komondjari", "Kompienga", "Kossi", "Koulpélogo", "Kouritenga", "Kourwéogo", "Léraba", "Loroum", "Mouhoun", "Nahouri", "Namentenga", "Nayala", "Noumbiel", "Oubritenga", "Oudalan", "Passoré", "Poni", "Sanguié", "Séno", "Sissili", "Soum", "Sourou", "Tapoa", "Tuy", "Yagha", "Yatenga", "Ziro", "Zondoma", "Zoundwéogo"
]

# División política de Burundi
divisiones_burundi = [
    "Bubanza", "Bujumbura Mairie", "Bujumbura Rural", "Bururi", "Cankuzo", "Cibitoke", "Gitega", "Karuzi", "Kayanza", "Kirundo", "Makamba", "Muramvya", "Muyinga", "Mwaro", "Ngozi", "Rumonge", "Rutana", "Ruyigi"
]

# División política de Costa de Marfil (Côte d'Ivoire)
divisiones_costa_marfil = [
    "Agnéby", "Bafing", "Bas-Sassandra", "Denguélé", "Gôh-Djiboua", "Lacs", "Lagunes", "Montagnes", "Sassandra-Marahoué", "Savanes", "Vallée du Bandama", "Worodougou", "Zanzan"
]

# División política de Cabo Verde
divisiones_cabo_verde = [
    "Boa Vista", "Brava", "Maio", "Mosteiros", "Paul", "Praia", "Ribeira Brava", "Ribeira Grande", "Ribeira Grande de Santiago", "Sal", "Santa Catarina", "Santa Catarina do Fogo", "Santa Cruz", "São Domingos", "São Filipe", "São Lourenço dos Órgãos", "São Miguel", "São Salvador do Mundo", "São Vicente", "Tarrafal", "Tarrafal de São Nicolau"
]

# División política de Camboya
divisiones_camboya = [
    "Banteay Meanchey", "Battambang", "Kampong Cham", "Kampong Chhnang", "Kampong Speu", "Kampong Thom", "Kampot", "Kandal", "Kep", "Koh Kong", "Kratié", "Mondulkiri", "Oddar Meanchey", "Pailin", "Phnom Penh", "Preah Sihanouk", "Preah Vihear", "Pursat", "Ratanakiri", "Siem Reap", "Stung Treng", "Svay Rieng", "Takéo", "Tbong Khmum"
]

# División política de Camerún
divisiones_camerun = [
    "Adamaoua", "Centro", "Este", "Extremo Norte", "Litoral", "Norte", "Noroeste", "Oeste", "Sur", "Suroeste"
]

# División política de Canadá
divisiones_canada = [
    "Alberta", "Columbia Británica", "Isla del Príncipe Eduardo", "Manitoba", "Nueva Brunswick", "Terranova y Labrador", "Nueva Escocia", "Nunavut", "Ontario", "Quebec", "Saskatchewan", "Territorios del Noroeste", "Yukón"
]

# División política de República Centroafricana
divisiones_centroafricana = [
    "Bamingui-Bangoran", "Bangui", "Basse-Kotto", "Haut-Mbomou", "Haute-Kotto", "Kémo", "Lobaye", "Mambéré-Kadéï", "Mbomou", "Nana-Grébizi", "Nana-Mambéré", "Ombella-M'Poko", "Ouaka", "Ouham", "Ouham-Pendé", "Sangha-Mbaéré", "Vakaga"
]

# División política de Chad
divisiones_chad = [
    "Bahr el Gazel", "Batha", "Borkou-Ennedi-Tibesti", "Chari-Baguirmi", "Ennedi-Est", "Ennedi-Ouest", "Guera", "Hadjer-Lamis", "Kanem", "Lac", "Logone Occidental", "Logone Oriental", "Mandoul", "Mayo-Kebbi Este", "Mayo-Kebbi Oeste", "Moyen-Chari", "N'Djaména", "Ouaddaï", "Salamat", "Sila", "Tandjilé", "Tibesti", "Ville de N'Djaména", "Wadi Fira"
]

# División política de Chile
divisiones_chile = [
    "Región de Arica y Parinacota", "Región de Tarapacá", "Región de Antofagasta", "Región de Atacama", "Región de Coquimbo", "Región de Valparaíso", "Región del Libertador General Bernardo O'Higgins", "Región del Maule", "Región de Ñuble", "Región del Biobío", "Región de La Araucanía", "Región de Los Ríos", "Región de Los Lagos", "Región de Aysén del General Carlos Ibáñez del Campo", "Región de Magallanes y de la Antártica Chilena", "Región Metropolitana de Santiago"
]

# División política de China
divisiones_china = [
    "Anhui", "Fujian", "Gansu", "Guangdong", "Guizhou", "Hainan", "Hebei", "Heilongjiang", "Henan", "Hubei", "Hunan", "Jiangsu", "Jiangxi", "Jilin", "Liaoning", "Qinghai", "Shaanxi", "Shandong", "Shanghai", "Shanxi", "Sichuan", "Tianjin", "Tibet", "Xinjiang", "Yunnan", "Zhejiang", "Chongqing", "Guangxi", "Inner Mongolia", "Ningxia", "Hong Kong", "Macau", "Taiwan"
]

# División política de Colombia
divisiones_colombia = [
    "Amazonas", "Antioquia", "Arauca", "Atlántico", "Bolívar", "Boyacá", "Caldas", "Caquetá", "Casanare", "Cauca", "Cesar", "Chocó", "Córdoba", "Cundinamarca", "Guainía", "Guaviare", "Huila", "La Guajira", "Magdalena", "Meta", "Nariño", "Norte de Santander", "Putumayo", "Quindío", "Risaralda", "San Andrés y Providencia", "Santander", "Sucre", "Tolima", "Valle del Cauca", "Vaupés", "Vichada"
]

# División política de Comoras
divisiones_comoras = [
    "Gran Comora", "Anjouan", "Mohéli"
]

# División política de República del Congo
divisiones_congo = [
    "Bouenza", "Brazzaville", "Cuvette", "Cuvette-Ouest", "Kouilou", "Likouala", "Niari", "Plateaux", "Pool", "Sangha"
]

# División política de Costa Rica
divisiones_costa_rica = [
    "Alajuela", "Cartago", "Guanacaste", "Heredia", "Limón", "Puntarenas", "San José"
]

# División política de Croacia
divisiones_croacia = [
    "Bjelovar-Bilogora", "Brod-Posavina", "Dubrovnik-Neretva", "Istria", "Karlovac", "Koprivnica-Križevci", "Krapina-Zagorje", "Lika-Senj", "Međimurje", "Osijek-Baranja", "Požega-Slavonia", "Primorje-Gorski Kotar", "Šibenik-Knin", "Sisak-Moslavina", "Split-Dalmatia", "Varaždin", "Virovitica-Podravina", "Vukovar-Srijem", "Zadar", "Zagreb", "Zagreb County"
]

# División política de Cuba
divisiones_cuba = [
    "Artemisa", "Camagüey", "Ciego de Ávila", "Cienfuegos", "Granma", "Guantánamo", "Holguín", "Isla de la Juventud", "La Habana", "Las Tunas", "Matanzas", "Mayabeque", "Pinar del Río", "Sancti Spíritus", "Santiago de Cuba", "Villa Clara"
]

# División política de Chipre
divisiones_chipre = [
    "Famagusta", "Kyrenia", "Larnaca", "Limassol", "Nicosia", "Paphos"
]

# División política de República Checa
divisiones_republica_checa = [
    "Bohemia Central", "Hradec Králové", "Karlovy Vary", "Liberec", "Moravia-Silesia", "Olomouc", "Pardubice", "Pilsen", "Prague", "South Bohemia", "South Moravia", "Ústí nad Labem", "Vysočina", "Zlín"
]

# División política de República Dominicana
divisiones_republica_dominicana = [
    "Azua", "Baoruco", "Barahona", "Dajabón", "Distrito Nacional", "Duarte", "Elías Piña", "El Seibo", "Espaillat", "Hato Mayor", "Hermanas Mirabal", "Independencia", "La Altagracia", "La Romana", "La Vega", "María Trinidad Sánchez", "Monseñor Nouel", "Monte Cristi", "Monte Plata", "Pedernales", "Peravia", "Puerto Plata", "Samaná", "Sánchez Ramírez", "San Cristóbal", "San José de Ocoa", "San Juan", "San Pedro de Macorís", "Santiago", "Santiago Rodríguez", "Santo Domingo", "Valverde"
]

# División política de Timor-Leste
divisiones_timor_leste = [
    "Aileu", "Ainaro", "Baucau", "Bobonaro", "Cova Lima", "Dili", "Ermera", "Lautém", "Liquiçá", "Manatuto", "Manufahi", "Oecusse", "Viqueque"
]

# División política de Ecuador
divisiones_ecuador = [
    "Azuay", "Bolívar", "Cañar", "Carchi", "Chimborazo", "Cotopaxi", "El Oro", "Esmeraldas", "Galápagos", "Guayas", "Imbabura", "Loja", "Los Ríos", "Manabí", "Morona Santiago", "Napo", "Orellana", "Pastaza", "Pichincha", "Santa Elena", "Santo Domingo de los Tsáchilas", "Sucumbíos", "Tungurahua", "Zamora-Chinchipe"
]

# División política de Egipto
divisiones_egipto = [
    "Aswan", "Alexandria", "Asyut", "Beheira", "Beni Suef", "Cairo", "Dakahlia", "Damietta", "Faiyum", "Gharbia", "Giza", "Ismailia", "Kafr el-Sheikh", "Luxor", "Matrouh", "Minya", "Monufia", "New Valley", "North Sinai", "Port Said", "Qalyubia", "Qena", "Red Sea", "Sharqia", "Sohag", "South Sinai", "Suez"
]

# División política de El Salvador
divisiones_el_salvador = [
    "Ahuachapán", "Cabañas", "Chalatenango", "Cuscatlán", "La Libertad", "La Paz", "La Unión", "Morazán", "San Miguel", "San Salvador", "San Vicente", "Santa Ana", "Sonsonate", "Usulután"
]

# División política de Guinea Ecuatorial
divisiones_guinea_ecuatorial = [
    "Annobón", "Bioko Norte", "Bioko Sur", "Centro Sur", "Kié-Ntem", "Litoral", "Wele-Nzas"
]

# División política de Eritrea
divisiones_eritrea = [
    "Anseba", "Debub", "Debub-Keih-Bahri", "Gash-Barka", "Maekel", "Semien-Keih-Bahri"
]

# División política de Estonia
divisiones_estonia = [
    "Harju", "Hiiu", "Ida-Viru", "Järva", "Jõgeva", "Lääne", "Lääne-Viru", "Pärnu", "Põlva", "Rapla", "Saare", "Tartu", "Valga", "Viljandi", "Võru"
]

# División política de Eswatini (Suazilandia)
divisiones_eswatini = [
    "Hhohho", "Lubombo", "Manzini", "Shiselweni"
]

# División política de Etiopía
divisiones_etiopia = [
    "Afar", "Amhara", "Benishangul-Gumuz", "Dire Dawa", "Gambela", "Harari", "Oromia", "Sidama", "Somali", "Southern Nations, Nationalities, and Peoples' Region", "Tigray"
]

# División política de Fiji
divisiones_fiji = [
    "Central", "Eastern", "Northern", "Rotuma", "Western"
]

# División política de Finlandia
divisiones_finlandia = [
    "Åland", "Central Finland", "Kainuu", "Kymenlaakso", "Lapland", "North Karelia", "Northern Ostrobothnia", "Northern Savo", "Ostrobothnia", "Päijät-Häme", "Pirkanmaa", "Satakunta", "South Karelia", "Southern Ostrobothnia", "Southern Savo", "Uusimaa", "Varsinais-Suomi"
]

# División política de Francia
divisiones_francia = [
    "Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Brittany", "Centre-Val de Loire", "Corsica", "Grand Est", "Hauts-de-France", "Île-de-France", "Normandy", "Nouvelle-Aquitaine", "Occitanie", "Pays de la Loire", "Provence-Alpes-Côte d'Azur"
]

# División política de Gabón
divisiones_gabon = [
    "Estuaire", "Haut-Ogooué", "Moyen-Ogooué", "Ngounié", "Nyanga", "Ogooué-Ivindo", "Ogooué-Lolo", "Ogooué-Maritime", "Woleu-Ntem"
]

# División política de Gambia
divisiones_gambia = [
    "Banjul", "Central River", "Lower River", "North Bank", "Upper River", "West Coast"
]

# División política de Georgia
divisiones_georgia = [
    "Adjara", "Guria", "Imereti", "Kakheti", "Kvemo Kartli", "Mtskheta-Mtianeti", "Racha-Lechkhumi and Kvemo Svaneti", "Samegrelo-Zemo Svaneti", "Samtskhe-Javakheti", "Shida Kartli", "Tbilisi"
]

# División política de Alemania
divisiones_alemania = [
    "Baden-Württemberg", "Bavaria", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hesse", "Lower Saxony", "Mecklenburg-Vorpommern", "North Rhine-Westphalia", "Rhineland-Palatinate", "Saarland", "Saxony", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"
]

# División política de Ghana
divisiones_ghana = [
    "Ahafo", "Ashanti", "Bono", "Bono East", "Central", "Eastern", "Greater Accra", "North East", "Northern", "Oti", "Savannah", "Upper East", "Upper West", "Volta", "Western", "Western North"
]

# División política de Grecia
divisiones_grecia = [
    "Attica", "Central Greece", "Central Macedonia", "Crete", "East Macedonia and Thrace", "Epirus", "Ionian Islands", "North Aegean", "Peloponnese", "South Aegean", "Thessaly", "West Greece", "West Macedonia"
]

# División política de Granada
divisiones_granada = [
    "Saint Andrew", "Saint David", "Saint George", "Saint John", "Saint Mark", "Saint Patrick"
]

# División política de Guatemala
divisiones_guatemala = [
    "Alta Verapaz", "Baja Verapaz", "Chimaltenango", "Chiquimula", "El Progreso", "Escuintla", "Guatemala", "Huehuetenango", "Izabal", "Jalapa", "Jutiapa", "Petén", "Quetzaltenango", "Quiché", "Retalhuleu", "Sacatepéquez", "San Marcos", "Santa Rosa", "Sololá", "Suchitepéquez", "Totonicapán", "Zacapa"
]

# División política de Guinea
divisiones_guinea = [
    "Beyla", "Boffa", "Boké", "Conakry", "Coyah", "Dabola", "Dalaba", "Dinguiraye", "Dubréka", "Faranah", "Forécariah", "Fria", "Gaoual", "Guékédou", "Kankan", "Kérouané", "Kindia", "Kissidougou", "Koubia", "Koundara", "Kouroussa", "Lélouma", "Lola", "Macenta", "Mali", "Mamou", "Mandiana", "Nzérékoré", "Pita", "Siguiri", "Télimélé", "Tougué", "Yomou"
]

# División política de Guinea-Bisáu
divisiones_guinea_bisau = [
    "Bafatá", "Biombo", "Bissau Autonomous Sector", "Bolama", "Cacheu", "Gabú", "Leste", "Norte", "Oio", "Quinara", "Sul", "Tombali"
]

# División política de Guyana
divisiones_guyana = [
    "Barima-Waini", "Cuyuni-Mazaruni", "Demerara-Mahaica", "East Berbice-Corentyne", "Essequibo Islands-West Demerara", "Mahaica-Berbice", "Pomeroon-Supenaam", "Potaro-Siparuni", "Upper Demerara-Berbice", "Upper Takutu-Upper Essequibo"
]

# División política de Haití
divisiones_haiti = [
    "Artibonite", "Centre", "Grand'Anse", "Nippes", "Nord", "Nord-Est", "Nord-Ouest", "Ouest", "Sud", "Sud-Est"
]

# División política de Santa Sede
divisiones_santa_sede = [
    "Vatican City"
]

# División política de Honduras
divisiones_honduras = [
    "Atlántida", "Choluteca", "Colón", "Comayagua", "Copán", "Cortés", "El Paraíso", "Francisco Morazán", "Gracias a Dios", "Intibucá", "Islas de la Bahía", "La Paz", "Lempira", "Ocotepeque", "Olancho", "Santa Bárbara", "Valle", "Yoro"
]

# División política de Hungría
divisiones_hungria = [
    "Bács-Kiskun", "Baranya", "Békés", "Borsod-Abaúj-Zemplén", "Csongrád-Csanád", "Fejér", "Győr-Moson-Sopron", "Hajdú-Bihar", "Heves", "Jász-Nagykun-Szolnok", "Komárom-Esztergom", "Nógrád", "Pest", "Somogy", "Szabolcs-Szatmár-Bereg", "Tolna", "Vas", "Veszprém", "Zala"
]

# División política de Islandia
divisiones_islandia = [
    "Capital Region", "East", "Northeast", "Northwest", "South", "Southern Peninsula", "Westfjords", "West"
]

# División política de India
divisiones_india = [
    "Andaman and Nicobar Islands", "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chandigarh", "Chhattisgarh", "Dadra and Nagar Haveli and Daman and Diu", "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Jharkhand", "Karnataka", "Kerala", "Ladakh", "Lakshadweep", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Puducherry", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"
]

# División política de Indonesia
divisiones_indonesia = [
    "Aceh", "Bali", "Bangka Belitung Islands", "Banten", "Bengkulu", "Central Java", "Central Kalimantan", "Central Sulawesi", "East Java", "East Kalimantan", "East Nusa Tenggara", "Gorontalo", "Jakarta", "Jambi", "Lampung", "Maluku", "North Kalimantan", "North Maluku", "North Sulawesi", "North Sumatra", "Papua", "Riau", "Riau Islands", "South Kalimantan", "South Sulawesi", "South Sumatra", "Southeast Sulawesi", "West Java", "West Kalimantan", "West Nusa Tenggara", "West Papua", "West Sulawesi", "West Sumatra", "Yogyakarta"
]

# División política de Irán
divisiones_iran = [
    "Alborz", "Ardabil", "Bushehr", "Chaharmahal and Bakhtiari", "East Azerbaijan", "Fars", "Gilan", "Golestan", "Hamadan", "Hormozgan", "Ilam", "Isfahan", "Kerman", "Kermanshah", "Khuzestan", "Kohgiluyeh and Boyer-Ahmad", "Kurdistan", "Lorestan", "Markazi", "Mazandaran", "North Khorasan", "Qazvin", "Qom", "Razavi Khorasan", "Semnan", "Sistan and Baluchestan", "South Khorasan", "Tehran", "West Azerbaijan", "Yazd", "Zanjan"
]

# División política de Iraq
divisiones_irak = [
    "Al-Anbar", "Al-Qadisiyyah", "Babylon", "Baghdad", "Basra", "Dhi Qar", "Diyala", "Dohuk", "Erbil", "Karbala", "Kirkuk", "Maysan", "Muthanna", "Najaf", "Nineveh", "Saladin", "Sulaymaniyah", "Wasit"
]

# División política de Irlanda
divisiones_irlanda = [
    "Carlow", "Cavan", "Clare", "Cork", "Donegal", "Dublin", "Galway", "Kerry", "Kildare", "Kilkenny", "Laois", "Leitrim", "Limerick", "Longford", "Louth", "Mayo", "Meath", "Monaghan", "Offaly", "Roscommon", "Sligo", "Tipperary", "Waterford", "Westmeath", "Wexford", "Wicklow"
]

# División política de Israel
divisiones_israel = [
    "Central District", "Haifa District", "Jerusalem District", "Northern District", "Southern District", "Tel Aviv District", "Judea and Samaria Area"
]

# División política de Italia
divisiones_italia = [
    "Abruzzo", "Aosta Valley", "Apulia", "Basilicata", "Calabria", "Campania", "Emilia-Romagna", "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardy", "Marche", "Molise", "Piedmont", "Sardinia", "Sicily", "Trentino-Alto Adige", "Tuscany", "Umbria", "Veneto"
]

# División política de Costa de Marfil
divisiones_costa_de_marfil = [
    "Agnéby-Tiassa", "Bafing", "Bas-Sassandra", "Denguélé", "Dix-Huit Montagnes", "Fromager", "Haut-Sassandra", "Lacs", "Lagunes", "Marahoué", "Moyen-Cavally", "Moyen-Comoé", "N'zi-Comoé", "Savanes", "Sud-Bandama", "Sud-Comoé", "Vallée du Bandama", "Worodougou", "Zanzan"
]

# División política de Jamaica
divisiones_jamaica = [
    "Clarendon", "Hanover", "Kingston", "Manchester", "Portland", "Saint Andrew", "Saint Ann", "Saint Catherine", "Saint Elizabeth", "Saint James", "Saint Mary", "Saint Thomas", "Trelawny", "Westmoreland"
]

# División política de Japón
divisiones_japon = [
    "Aichi", "Akita", "Aomori", "Chiba", "Ehime", "Fukui", "Fukuoka", "Fukushima", "Gifu", "Gunma", "Hiroshima", "Hokkaido", "Hyogo", "Ibaraki", "Ishikawa", "Iwate", "Kagawa", "Kagoshima", "Kanagawa", "Kochi", "Kumamoto", "Kyoto", "Mie", "Miyagi", "Miyazaki", "Nagano", "Nagasaki", "Nara", "Niigata", "Oita", "Okayama", "Okinawa", "Osaka", "Saga", "Saitama", "Shiga", "Shimane", "Shizuoka", "Tochigi", "Tokushima", "Tokyo", "Tottori", "Toyama", "Wakayama", "Yamagata", "Yamaguchi", "Yamanashi"
]

# División política de Jordania
divisiones_jordania = [
    "Ajloun", "Amman", "Aqaba", "Balqa", "Irbid", "Jerash", "Karak", "Ma'an", "Madaba", "Mafraq", "Tafilah", "Zarqa"
]

# División política de Kazajistán
divisiones_kazajistan = [
    "Aktobe", "Almaty", "Aqmola", "Aqtöbe", "Atyrau", "East Kazakhstan", "Mangystau", "North Kazakhstan", "Nur-Sultan", "Pavlodar", "Shymkent", "Turkestan", "West Kazakhstan"
]

# División política de Kenia
divisiones_kenia = [
    "Baringo", "Bomet", "Bungoma", "Busia", "Elgeyo-Marakwet", "Embu", "Garissa", "Homa Bay", "Isiolo", "Kajiado", "Kakamega", "Kericho", "Kiambu", "Kilifi", "Kirinyaga", "Kisii", "Kisumu", "Kitui", "Kwale", "Laikipia", "Lamu", "Machakos", "Makueni", "Mandera", "Marsabit", "Meru", "Migori", "Mombasa", "Murang'a", "Nairobi City", "Nakuru", "Nandi", "Narok", "Nyamira", "Nyandarua", "Nyeri", "Samburu", "Siaya", "Taita-Taveta", "Tana River", "Tharaka-Nithi", "Trans Nzoia", "Turkana", "Uasin Gishu", "Vihiga", "Wajir", "West Pokot"
]

# División política de Kiribati
divisiones_kiribati = [
    "Gilbert Islands", "Line Islands", "Phoenix Islands"
]

# División política de Kuwait
divisiones_kuwait = [
    "Al Ahmadi", "Al Asimah", "Al Farwaniyah", "Al Jahra", "Hawalli", "Mubarak Al-Kabeer"
]

# División política de Kirguistán
divisiones_kirguistan = [
    "Batken", "Bishkek", "Chuy", "Jalal-Abad", "Naryn", "Osh", "Talas", "Ysyk-Köl"
]

# División política de Laos
divisiones_laos = [
    "Attapeu", "Bokeo", "Bolikhamxai", "Champasak", "Houaphanh", "Khammouane", "Luang Namtha", "Luang Prabang", "Oudomxay", "Phongsaly", "Salavan", "Savannakhet", "Sainyabuli", "Sekong", "Vientiane", "Vientiane Prefecture", "Xaisomboun", "Xekong", "Xiangkhouang"
]

# División política de Letonia
divisiones_letonia = [
    "Aizkraukle", "Alūksne", "Balvi", "Bauska", "Cēsis", "Daugavpils", "Dobele", "Gulbene", "Jēkabpils", "Jelgava", "Jurmala", "Krāslava", "Kuldīga", "Liepāja", "Limbaži", "Ludza", "Madona", "Ogre", "Preiļi", "Rēzekne", "Riga", "Saldus", "Talsi", "Tukums", "Valka", "Valmiera", "Ventspils"
]

# División política de Líbano
divisiones_libano = [
    "Beirut", "Beqaa", "Mount Lebanon", "Nabatieh", "North Governorate", "South Governorate"
]

# División política de Lesoto
divisiones_lesoto = [
    "Berea", "Butha-Buthe", "Leribe", "Mafeteng", "Maseru", "Mohale's Hoek", "Mokhotlong", "Qacha's Nek", "Quthing", "Thaba-Tseka"
]

# División política de Liberia
divisiones_liberia = [
    "Bomi", "Bong", "Gbarpolu", "Grand Bassa", "Grand Cape Mount", "Grand Gedeh", "Grand Kru", "Lofa", "Margibi", "Maryland", "Montserrado", "Nimba", "River Cess", "River Gee", "Sinoe"
]

# División política de Libia
divisiones_libia = [
    "Al Wahat", "Benghazi", "Butnan", "Derna", "Ghat", "Jabal al Akhdar", "Jabal al Gharbi", "Jafara", "Jufra", "Kufra", "Marj", "Misrata", "Murqub", "Nalut", "Nuwaidrat", "Sabha", "Sirte", "Tripoli", "Wadi al Hayaa", "Wadi al Shatii", "Zawiya"
]

# División política de Liechtenstein
divisiones_liechtenstein = [
    "Balzers", "Eschen", "Gamprin", "Mauren", "Planken", "Ruggell", "Schaan", "Schellenberg", "Triesen", "Triesenberg", "Vaduz"
]

# División política de Lituania
divisiones_lituania = [
    "Alytus County", "Kaunas County", "Klaipėda County", "Marijampolė County", "Panevėžys County", "Šiauliai County", "Tauragė County", "Telšiai County", "Utena County", "Vilnius County"
]

# División política de Luxemburgo
divisiones_luxemburgo = [
    "Capellen", "Clervaux", "Diekirch", "Echternach", "Esch-sur-Alzette", "Grevenmacher", "Luxembourg", "Mersch", "Redange", "Remich", "Vianden", "Wiltz"
]

# División política de Madagascar
divisiones_madagascar = [
    "Alaotra-Mangoro", "Amoron'i Mania", "Analamanga", "Analanjirofo", "Androy", "Anosy", "Atsimo-Andrefana", "Atsimo-Atsinanana", "Atsinanana", "Betsiboka", "Boeny", "Bongolava", "Diana", "Haute Matsiatra", "Ihorombe", "Itasy", "Melaky", "Menabe", "Sava", "Sofia", "Vakinankaratra", "Vatovavy-Fitovinany"
]

# División política de Malawi
divisiones_malawi = [
    "Central Region", "Northern Region", "Southern Region"
]

# División política de Malasia
divisiones_malasia = [
    "Johor", "Kedah", "Kelantan", "Kuala Lumpur", "Labuan", "Melaka", "Negeri Sembilan", "Pahang", "Penang", "Perak", "Perlis", "Putrajaya", "Sabah", "Sarawak", "Selangor", "Terengganu"
]

# División política de Maldivas
divisiones_maldivas = [
    "Alif Alif Atoll", "Alif Dhaal Atoll", "Baa Atoll", "Dhaalu Atoll", "Faafu Atoll", "Gaafu Alif Atoll", "Gaafu Dhaalu Atoll", "Gnaviyani Atoll", "Haa Alif Atoll", "Haa Dhaalu Atoll", "Kaafu Atoll", "Lhaviyani Atoll", "Malé", "Meemu Atoll", "Noonu Atoll", "Raa Atoll", "Seenu Atoll", "Shaviyani Atoll", "Thaa Atoll", "Vaavu Atoll"
]

# División política de Mali
divisiones_mali = [
    "Bamako", "Gao", "Kayes", "Kidal", "Koulikoro", "Ménaka", "Mopti", "Ségou", "Sikasso", "Taoudénit", "Tombouctou"
]

# División política de Malta
divisiones_malta = [
    "Gozo and Comino", "Northern Region", "Southern Region"
]

# División política de Islas Marshall
divisiones_islas_marshall = [
    "Ailinglaplap Atoll", "Ailuk Atoll", "Arno Atoll", "Aur Atoll", "Bikar Atoll", "Bikini Atoll", "Ebon Atoll", "Enewetak Atoll", "Jabat Island", "Jaluit Atoll", "Kili Island", "Kwajalein Atoll", "Lae Atoll", "Lib Island", "Likiep Atoll", "Majuro Atoll", "Maloelap Atoll", "Mejit Island", "Namdrik Atoll", "Namu Atoll", "Rong Atoll", "Taka Atoll", "Ujae Atoll", "Ujelang Atoll", "Utrik Atoll", "Wotho Atoll", "Wotje Atoll"
]

# División política de Mauritania
divisiones_mauritania = [
    "Adrar", "Assaba", "Brakna", "Dakhlet Nouadhibou", "Gorgol", "Guidimaka", "Hodh Ech Chargui", "Hodh El Gharbi", "Inchiri", "Nouakchott", "Tagant", "Tiris Zemmour", "Trarza"
]

# División política de Mauricio
divisiones_mauricio = [
    "Black River", "Flacq", "Grand Port", "Moka", "Pamplemousses", "Plaines Wilhems", "Port Louis", "Rivière du Rempart", "Rodrigues", "Savanne"
]

# División política de México
divisiones_mexico = [
    "Aguascalientes", "Baja California", "Baja California Sur", "Campeche", "Chiapas", "Chihuahua", "Coahuila", "Colima", "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "Mexico City", "Mexico State", "Michoacán", "Morelos", "Nayarit", "Nuevo Leon", "Oaxaca", "Puebla", "Querétaro", "Quintana Roo", "San Luis Potosi", "Sinaloa", "Sonora", "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatan", "Zacatecas"
]

# División política de Micronesia
divisiones_micronesia = [
    "Chuuk", "Kosrae", "Pohnpei", "Yap"
]

# División política de Moldavia
divisiones_moldavia = [
    "Bălți", "Cahul", "Călărași", "Cimișlia", "Criuleni", "Dondușeni", "Drochia", "Dubăsari", "Edineț", "Fălești", "Florești", "Găgăuzia", "Hîncești", "Ialoveni", "Nisporeni", "Ocnița", "Orhei", "Rezina", "Rîșcani", "Sîngerei", "Soroca", "Strășeni", "Taraclia", "Telenesti", "Ungheni"
]

# División política de Mónaco
divisiones_monaco = [
    "Fontvieille", "La Condamine", "Monaco-Ville", "Monte-Carlo"
]

# División política de Mongolia
divisiones_mongolia = [
    "Arkhangai", "Bayan-Ölgii", "Bayankhongor", "Bulgan", "Darkhan-Uul", "Dornod", "Dornogovi", "Dundgovi", "Govi-Altai", "Khentii", "Khovd", "Khövsgöl", "Orkhon", "Övörkhangai", "Selenge", "Sükhbaatar", "Töv", "Uvs"
]

# División política de Montenegro
divisiones_montenegro = [
    "Andrijevica", "Bar", "Berane", "Bijelo Polje", "Budva", "Cetinje", "Danilovgrad", "Gusinje", "Kolašin", "Kotor", "Mojkovac", "Nikšić", "Plav", "Pljevlja", "Plužine", "Podgorica", "Rožaje", "Tivat", "Ulcinj", "Žabljak"
]

# División política de Marruecos
divisiones_marruecos = [
    "Beni Mellal-Khenifra", "Casablanca-Settat", "Dakhla-Oued Ed-Dahab", "Drâa-Tafilalet", "Fès-Meknès", "Guelmim-Oued Noun", "Laâyoune-Sakia El Hamra", "Marrakech-Safi", "Oriental", "Rabat-Salé-Kénitra", "Souss-Massa", "Tanger-Tétouan-Al Hoceima"
]

# División política de Mozambique
divisiones_mozambique = [
    "Cabo Delgado", "Gaza", "Inhambane", "Manica", "Maputo", "Nampula", "Niassa", "Sofala", "Tete", "Zambézia"
]

# División política de Myanmar (Birmania)
divisiones_myanmar = [
    "Ayeyarwady", "Bago", "Chin", "Kachin", "Kayah", "Kayin", "Magway", "Mandalay", "Mon", "Naypyidaw Union Territory", "Rakhine", "Sagaing", "Shan", "Tanintharyi", "Yangon"
]

# División política de Namibia
divisiones_namibia = [
    "Erongo", "Hardap", "Karas", "Kavango East", "Kavango West", "Khomas", "Kunene", "Ohangwena", "Omaheke", "Omusati", "Oshana", "Oshikoto", "Otjozondjupa", "Zambezi"
]

# División política de Nauru
divisiones_nauru = [
    "Aiwo", "Anabar", "Anetan", "Anibare", "Baiti", "Boe", "Buada", "Denigomodu", "Ewa", "Ijuw", "Meneng", "Nibok", "Uaboe", "Yaren"
]

# División política de Nepal
divisiones_nepal = [
    "Bagmati", "Bheri", "Dhaulagiri", "Gandaki", "Janakpur", "Karnali", "Kosi", "Lumbini", "Mechi", "Narayani", "Rapti", "Sagarmatha", "Sudurpashchim"
]

# División política de los Países Bajos
divisiones_paises_bajos = [
    "Drenthe", "Flevoland", "Friesland", "Gelderland", "Groningen", "Limburg", "Noord-Brabant", "Noord-Holland", "Overijssel", "Utrecht", "Zeeland", "Zuid-Holland"
]

# División política de Nueva Zelanda
divisiones_nueva_zelanda = [
    "Auckland", "Bay of Plenty", "Canterbury", "Chatham Islands Territory", "Gisborne", "Hawke's Bay", "Manawatu-Wanganui", "Marlborough", "Nelson", "Northland", "Otago", "Southland", "Taranaki", "Tasman", "Waikato", "Wellington", "West Coast"
]

# División política de Nicaragua
divisiones_nicaragua = [
    "Atlántico Norte", "Atlántico Sur", "Boaco", "Carazo", "Chinandega", "Chontales", "Estelí", "Granada", "Jinotega", "León", "Madriz", "Managua", "Masaya", "Matagalpa", "Nueva Segovia", "Río San Juan", "Rivas"
]

# División política de Níger
divisiones_niger = [
    "Agadez", "Diffa", "Dosso", "Maradi", "Tahoua", "Tillabéri", "Zinder"
]

# División política de Nigeria
divisiones_nigeria = [
    "Abia", "Adamawa", "Akwa Ibom", "Anambra", "Bauchi", "Bayelsa", "Benue", "Borno", "Cross River", "Delta", "Ebonyi", "Edo", "Ekiti", "Enugu", "Federal Capital Territory", "Gombe", "Imo", "Jigawa", "Kaduna", "Kano", "Katsina", "Kebbi", "Kogi", "Kwara", "Lagos", "Nasarawa", "Niger", "Ogun", "Ondo", "Osun", "Oyo", "Plateau", "Rivers", "Sokoto", "Taraba", "Yobe", "Zamfara"
]

# División política de Corea del Norte
divisiones_corea_del_norte = [
    "Chagang", "Hamgyongbuk", "Hamgyongnam", "Hwanghaebuk", "Hwanghaenam", "Kangwon", "Pyonganbuk", "Pyongannam", "Rason", "Ryanggang"
]

# División política de Corea del Sur
divisiones_corea_del_sur = [
    "Chungcheongbuk", "Chungcheongnam", "Gangwon", "Gyeonggi", "Gyeongsangbuk", "Gyeongsangnam", "Incheon", "Jeju", "Jeollabuk", "Jeollanam", "Sejong", "Seoul", "Ulsan"
]

# División política de Macedonia del Norte
divisiones_macedonia_del_norte = [
    "East", "Northeast", "Skopje", "Southeast", "Southwest", "Vardar"
]

# División política de Moldavia
divisiones_moldavia = [
    "Anenii Noi", "Bălți", "Bender", "Briceni", "Cahul", "Cantemir", "Călărași", "Căușeni", "Chișinău", "Cimișlia", "Criuleni", "Dondușeni", "Drochia", "Dubăsari", "Edineț", "Fălești", "Floresti", "Glodeni", "Hîncești", "Ialoveni", "Nisporeni", "Ocnița", "Orhei", "Rezina", "Rîșcani", "Sîngerei", "Soroca", "Strășeni", "Taraclia", "Telenesti", "Ungheni"
]

# División política de Omán
divisiones_oman = [
    "Ad Dakhiliyah", "Ad Dhahirah", "Al Batinah North", "Al Batinah South", "Al Buraimi", "Al Wusta", "Ash Sharqiyah North", "Ash Sharqiyah South", "Dhofar", "Musandam", "Muscat"
]

# División política de Pakistán
divisiones_pakistan = [
    "Azad Jammu and Kashmir", "Balochistan", "Gilgit-Baltistan", "Islamabad Capital Territory", "Khyber Pakhtunkhwa", "Punjab", "Sindh"
]

# División política de Palau
divisiones_palau = [
    "Aimeliik", "Airai", "Angaur", "Hatohobei", "Kayangel", "Koror", "Melekeok", "Ngaraard", "Ngarchelong", "Ngardmau", "Ngatpang", "Ngchesar", "Ngeremlengui", "Ngiwal", "Peleliu", "Sonsorol"
]

# División política de Panamá
divisiones_panama = [
    "Bocas del Toro", "Coclé", "Colón", "Darién", "Emberá", "Herrera", "Kuna de Wargandí", "Kuna Yala", "Los Santos", "Ngäbe-Buglé", "Panamá", "Veraguas"
]

# División política de Papúa Nueva Guinea
divisiones_papua_nueva_guinea = [
    "Bougainville", "Central", "Chimbu", "East New Britain", "East Sepik", "Eastern Highlands", "Enga", "Gulf", "Hela", "Jiwaka", "Madang", "Manus", "Milne Bay", "Morobe", "National Capital", "New Ireland", "Oro", "Sandaun", "Southern Highlands", "West New Britain", "Western", "Western Highlands"
]

# División política de Paraguay
divisiones_paraguay = [
    "Alto Paraguay", "Alto Paraná", "Amambay", "Boquerón", "Caaguazú", "Caazapá", "Canindeyú", "Central", "Concepción", "Cordillera", "Guairá", "Itapúa", "Misiones", "Ñeembucú", "Paraguarí", "Presidente Hayes", "San Pedro"
]

# División política de Perú
divisiones_peru = [
    "Amazonas", "Ancash", "Apurímac", "Arequipa", "Ayacucho", "Cajamarca", "Callao", "Cusco", "Huancavelica", "Huánuco", "Ica", "Junín", "La Libertad", "Lambayeque", "Lima", "Loreto", "Madre de Dios", "Moquegua", "Pasco", "Piura", "Puno", "San Martín", "Tacna", "Tumbes", "Ucayali"
]

# División política de Filipinas
divisiones_filipinas = [
    "Abra", "Agusan del Norte", "Agusan del Sur", "Aklan", "Albay", "Antique", "Apayao", "Aurora", "Basilan", "Bataan", "Batanes", "Batangas", "Benguet", "Biliran", "Bohol", "Bukidnon", "Bulacan", "Cagayan", "Camarines Norte", "Camarines Sur", "Camiguin", "Capiz", "Catanduanes", "Cavite", "Cebu", "Cotabato", "Davao de Oro", "Davao del Norte", "Davao del Sur", "Davao Occidental", "Davao Oriental", "Dinagat Islands", "Eastern Samar", "Guimaras", "Ifugao", "Ilocos Norte", "Ilocos Sur", "Iloilo", "Isabela", "Kalinga", "La Union", "Laguna", "Lanao del Norte", "Lanao del Sur", "Leyte", "Maguindanao", "Marinduque", "Masbate", "Misamis Occidental", "Misamis Oriental", "Mountain Province", "Negros Occidental", "Negros Oriental", "Northern Samar", "Nueva Ecija", "Nueva Vizcaya", "Occidental Mindoro", "Oriental Mindoro", "Palawan", "Pampanga", "Pangasinan", "Quezon", "Quirino", "Rizal", "Romblon", "Samar", "Sarangani", "Siquijor", "Sorsogon", "South Cotabato", "Southern Leyte", "Sultan Kudarat", "Sulu", "Surigao del Norte", "Surigao del Sur", "Tarlac", "Tawi-Tawi", "Zambales", "Zamboanga del Norte", "Zamboanga del Sur", "Zamboanga Sibugay"
]

# División política de Polonia
divisiones_polonia = [
    "Greater Poland", "Kuyavian-Pomeranian", "Lesser Poland", "Łódź", "Lower Silesian", "Lubusz", "Masovian", "Opole", "Podkarpackie", "Podlaskie", "Pomeranian", "Silesian", "Subcarpathian", "Swietokrzyskie", "Warmian-Masurian", "West Pomeranian"
]

# División política de Portugal
divisiones_portugal = [
    "Aveiro", "Azores", "Beja", "Braga", "Bragança", "Castelo Branco", "Coimbra", "Évora", "Faro", "Guarda", "Leiria", "Lisbon", "Madeira", "Portalegre", "Porto", "Santarém", "Setúbal", "Viana do Castelo", "Vila Real", "Viseu"
]

# División política de Catar
divisiones_qatar = [
    "Ad Dawhah", "Al Daayen", "Al Khawr", "Al Wakrah", "Ar Rayyan", "Ash Shamal", "Umm Salal"
]

# División política de Rumania
divisiones_rumania = [
    "Alba", "Arad", "Argeș", "Bacău", "Bihor", "Bistrița-Năsăud", "Botoșani", "Brașov", "Brăila", "București", "Buzău", "Călărași", "Caraș-Severin", "Cluj", "Constanța", "Covasna", "Dâmbovița", "Dolj", "Galați", "Giurgiu", "Gorj", "Harghita", "Hunedoara", "Ialomița", "Iași", "Ilfov", "Maramureș", "Mehedinți", "Mureș", "Neamț", "Olt", "Prahova", "Sălaj", "Satu Mare", "Sibiu", "Suceava", "Teleorman", "Timiș", "Tulcea", "Vâlcea", "Vaslui", "Vrancea"
]

# División política de Rusia
divisiones_rusia = [
    "Adygea", "Altai", "Amur", "Arkhangelsk", "Astrakhan", "Bashkortostan", "Belgorod", "Bryansk", "Buryatia", "Chechnya", "Chelyabinsk", "Chukotka", "Chuvashia", "Dagestan", "Ingushetia", "Irkutsk", "Ivanovo", "Jewish Autonomous Region", "Kabardino-Balkaria", "Kaliningrad", "Kalmykia", "Kaluga", "Kamchatka", "Karachay-Cherkessia", "Karelia", "Kemerovo", "Khabarovsk", "Khakassia", "Khanty-Mansi", "Kirov", "Komi", "Koryak", "Kostroma", "Krasnodar", "Krasnoyarsk", "Kurgan", "Kursk", "Leningrad", "Lipetsk", "Magadan", "Mari El", "Mordovia", "Moscow", "Moscow (City)", "Murmansk", "Nenets", "Nizhny Novgorod", "North Ossetia-Alania", "Novgorod", "Novosibirsk", "Omsk", "Orenburg", "Oryol", "Penza", "Perm", "Primorsky", "Pskov", "Rostov", "Ryazan", "Sakha", "Sakhalin", "Samara", "Saratov", "Sevastopol (City)", "Smolensk", "Stavropol", "Sverdlovsk", "Tambov", "Tatarstan", "Tomsk", "Tula", "Tuva", "Tver", "Tyumen", "Udmurtia", "Ulyanovsk", "Vladimir", "Volgograd", "Vologda", "Voronezh", "Yamalo-Nenets", "Yaroslavl", "Zabaykalsky"
]

# División política de Ruanda
divisiones_ruanda = [
    "Eastern", "Kigali City", "Northern", "Southern", "Western"
]

# División política de San Cristóbal y Nieves
divisiones_san_cristobal_y_nieves = [
    "Christ Church Nichola Town", "Nevis", "Saint Anne Sandy Point", "Saint George Basseterre", "Saint George Gingerland", "Saint James Windward", "Saint John Capisterre", "Saint John Figtree", "Saint Mary Cayon", "Saint Paul Capisterre", "Saint Paul Charlestown", "Saint Peter Basseterre", "Saint Thomas Lowland", "Saint Thomas Middle Island", "Trinity Palmetto Point"
]

# División política de Santa Lucía
divisiones_santa_lucia = [
    "Anse la Raye", "Castries", "Choiseul", "Dauphin", "Dennery", "Gros Islet", "Laborie", "Micoud", "Praslin", "Soufrière", "Vieux Fort"
]

# División política de San Vicente y las Granadinas
divisiones_san_vicente_y_las_granadinas = [
    "Charlotte", "Grenadines", "Saint Andrew", "Saint David", "Saint George", "Saint Patrick"
]

# División política de Samoa
divisiones_samoa = [
    "A'ana", "Aiga-i-le-Tai", "Atua", "Fa'asaleleaga", "Gaga'emauga", "Gagaifomauga", "Palauli", "Satupa'itea", "Tuamasaga", "Va'a-o-Fonoti", "Vaisigano"
]

# División política de San Marino
divisiones_san_marino = [
    "Acquaviva", "Borgo Maggiore", "Chiesanuova", "Domagnano", "Faetano", "Fiorentino", "Montegiardino", "San Marino", "Serravalle"
]

# División política de Santo Tomé y Príncipe
divisiones_santo_tome_y_principe = [
    "Príncipe", "São Tomé"
]

# División política de Arabia Saudita
divisiones_arabia_saudita = [
    "Al Bahah", "Al Jawf", "Al Madinah", "Al Qasim", "Asir", "Eastern Province", "Ha'il", "Jazan", "Makkah", "Najran", "Northern Border", "Riyadh", "Tabuk"
]

# División política de Senegal
divisiones_senegal = [
    "Dakar", "Diourbel", "Fatick", "Kaffrine", "Kaolack", "Kédougou", "Kolda", "Louga", "Matam", "Saint-Louis", "Sédhiou", "Tambacounda", "Thiès", "Ziguinchor"
]

# División política de Serbia
divisiones_serbia = [
    "Belgrade", "Bor", "Braničevo", "Jablanica", "Kolubara", "Mačva", "Moravica", "Nišava", "Pčinja", "Pirot", "Podunavlje", "Pomoravlje", "Rasina", "Raška", "South Banat", "South Bačka", "Srem", "Toplica", "West Bačka", "Zaječar", "Zlatibor"
]

# División política de Seychelles
divisiones_seychelles = [
    "Anse aux Pins", "Anse Boileau", "Anse Etoile", "Anse Louis", "Anse Royale", "Baie Lazare", "Baie Sainte Anne", "Beau Vallon", "Bel Air", "Bel Ombre", "Cascade", "Glacis", "Grand'Anse", "Grand'Anse Mahe", "La Digue", "La Riviere Anglaise", "Les Mamelles", "Mont Buxton", "Mont Fleuri", "Plaisance", "Pointe La Rue", "Port Glaud", "Roche Caiman", "Saint Louis", "Takamaka"
]

# División política de Sierra Leona
divisiones_sierra_leona = [
    "Eastern", "Northern", "Southern", "Western"
]

# División política de Singapur
divisiones_singapur = [
    "Central Region", "East Region", "North Region", "North-East Region", "West Region"
]

# División política de Eslovaquia
divisiones_eslovaquia = [
    "Bratislava Region", "Košice Region", "Nitra Region", "Prešov Region", "Trenčín Region", "Trnava Region", "Žilina Region"
]

# División política de Eslovenia
divisiones_eslovenia = [
    "Ajdovščina", "Beltinci", "Bled", "Bohinj", "Borovnica", "Bovec", "Brda", "Brezovica", "Brežice", "Cankova", "Celje", "Cerklje na Gorenjskem", "Cerknica", "Cerkno", "Črenšovci", "Črna na Koroškem", "Črnomelj", "Destrnik", "Divaca", "Dobrepolje", "Dobrova-Polhov Gradec", "Dobrovnik", "Dol pri Ljubljani", "Dornava", "Dravograd", "Duplek", "Gorenja Vas-Poljane", "Gorišnica", "Gornja Radgona", "Gornji Grad", "Gornji Petrovci", "Grosuplje", "Hajdina", "Hoce-Slivnica", "Hodoš", "Horjul", "Hrastnik", "Hrpelje-Kozina", "Idrija", "Ig", "Ilirska Bistrica", "Ivancna Gorica", "Izola", "Jesenice", "Jezersko", "Juršinci", "Kamnik", "Kanal", "Kidricevo", "Kobarid", "Kobilje", "Kočevje", "Komen", "Koper", "Kostanjevica na Krki", "Kostel", "Kozje", "Kranj", "Kranjska Gora", "Kungota", "Kuzma", "Laško", "Lenart", "Lendava", "Litija", "Ljubljana", "Ljubno", "Ljutomer", "Log-Dragomer", "Logatec", "Loška Dolina", "Loški Potok", "Lovrenc na Pohorju", "Luče", "Majšperk", "Makole", "Maribor", "Markovci", "Medvode", "Menges", "Metlika", "Mezica", "Miklavž na Dravskem Polju", "Miren-Kostanjevica", "Mirna Pec", "Mislinja", "Mokronog-Trebelno", "Moravce", "Moravske Toplice", "Mozirje", "Murska Sobota", "Muta", "Naklo", "Nazarje", "Nova Gorica", "Novo Mesto", "Odranci", "Ormož", "Osilnica", "Pesnica", "Piran", "Pivka", "Podčetrtek", "Podlehnik", "Podvelka", "Polzela", "Postojna", "Prebold", "Preddvor", "Prevalje", "Ptuj", "Puconci", "Rače-Fram", "Radece", "Radenci", "Radlje ob Dravi", "Radovljica", "Ravne na Koroškem", "Ribnica", "Rogasevci", "Rogaška Slatina", "Rogatec", "Ruše", "Selnica ob Dravi", "Semič", "Sevnica", "Sežana", "Slovenj Gradec", "Slovenska Bistrica", "Slovenske Konjice", "Sodražica", "Solčava", "Središče ob Dravi", "Starše", "Straža", "Sveta Ana", "Sveti Andraž v Slovenskih Goricah", "Sveti Jurij ob Ščavnici", "Sveti Jurij v Slovenskih Goricah", "Sveti Tomaž", "Tabor", "Tišina", "Tolmin", "Trbovlje", "Trebnje", "Trnovska Vas", "Trzin", "Turnišče", "Velenje", "Velika Polana", "Velike Lašče", "Veržej", "Videm", "Vipava", "Vitanje", "Vodice", "Vojnik", "Vransko", "Vrhnika", "Vuzenica", "Zagorje ob Savi", "Zavrč", "Zreče", "Zagorje ob Savi"
]

# División política de Islas Salomón
divisiones_islas_salomon = [
    "Central", "Choiseul", "Guadalcanal", "Honiara", "Isabel", "Makira-Ulawa", "Malaita", "Rennell and Bellona", "Temotu", "Western"
]

# División política de Somalia
divisiones_somalia = [
    "Awdal", "Bakool", "Banaadir", "Bari", "Bay", "Galguduud", "Gedo", "Hiiraan", "Lower Juba", "Lower Shebelle", "Mudug", "Nugal", "Sanaag", "Sool", "Togdheer", "Woqooyi Galbeed"
]

# División política de Sudáfrica
divisiones_sudafrica = [
    "Eastern Cape", "Free State", "Gauteng", "KwaZulu-Natal", "Limpopo", "Mpumalanga", "North West", "Northern Cape", "Western Cape"
]

# División política de Sudán del Sur
divisiones_sudan_del_sur = [
    "Central Equatoria", "Eastern Equatoria", "Jonglei", "Lakes", "Northern Bahr el Ghazal", "Unity", "Upper Nile", "Warrap", "Western Bahr el Ghazal", "Western Equatoria"
]

# División política de España
divisiones_espana = [
    "Andalucía", "Aragón", "Asturias", "Canarias", "Cantabria", "Castilla y León", "Castilla-La Mancha", "Cataluña", "Ceuta (autonomous city)", "Extremadura", "Galicia", "Balearic Islands", "La Rioja", "Madrid (autonomous city)", "Melilla (autonomous city)", "Murcia", "Navarra", "Basque Country", "Valencia"
]

# División política de Sri Lanka
divisiones_sri_lanka = [
    "Central", "Eastern", "North Central", "Northern", "North Western", "Sabaragamuwa", "Southern", "Uva", "Western"
]

# División política de Sudán
divisiones_sudan = [
    "Blue Nile", "Central Darfur", "East Darfur", "Kassala", "Khartoum", "North Darfur", "North Kordofan", "Northern", "Red Sea", "River Nile", "Sennar", "South Darfur", "South Kordofan", "West Darfur", "West Kordofan", "White Nile"
]

# División política de Surinam
divisiones_surinam = [
    "Brokopondo", "Commewijne", "Coronie", "Marowijne", "Nickerie", "Para", "Paramaribo", "Saramacca", "Sipaliwini", "Wanica"
]

# División política de Suazilandia
divisiones_suazilandia = [
    "Hhohho", "Lubombo", "Manzini", "Shiselweni"
]

# División política de Suecia
divisiones_suecia = [
    "Blekinge", "Dalarna", "Gävleborg", "Gotland", "Halland", "Jämtland", "Jönköping", "Kalmar", "Kronoberg", "Norrbotten", "Örebro", "Östergötland", "Skåne", "Södermanland", "Stockholm", "Uppsala", "Värmland", "Västerbotten", "Västernorrland", "Västmanland", "Västra Götaland"
]

# División política de Suiza
divisiones_suiza = [
    "Aargau", "Appenzell Ausserrhoden", "Appenzell Innerrhoden", "Basel-Landschaft", "Basel-Stadt", "Bern", "Fribourg", "Geneva", "Glarus", "Graubünden", "Jura", "Lucerne", "Neuchâtel", "Nidwalden", "Obwalden", "Schaffhausen", "Schwyz", "Solothurn", "St. Gallen", "Thurgau", "Ticino", "Uri", "Valais", "Vaud", "Zug", "Zurich"
]

# División política de Siria
divisiones_siria = [
    "Al-Hasakah", "Al-Latakia", "Al-Quneitra", "Ar-Raqqah", "As-Suwayda", "Daraa", "Deir ez-Zor", "Dimashq", "Hama", "Homs", "Idlib", "Rif Dimashq", "Tartus"
]

# División política de Tayikistán
divisiones_tayikistan = [
    "Dushanbe (City)", "Khatlon", "Sughd"
]

# División política de Tanzania
divisiones_tanzania = [
    "Arusha", "Dar es Salaam", "Dodoma", "Geita", "Iringa", "Kagera", "Katavi", "Kigoma", "Kilimanjaro", "Lindi", "Manyara", "Mara", "Mbeya", "Morogoro", "Mtwara", "Mwanza", "Njombe", "Pemba North", "Pemba South", "Pwani", "Rukwa", "Ruvuma", "Shinyanga", "Simiyu", "Singida", "Tabora", "Tanga", "Zanzibar Central/South", "Zanzibar North", "Zanzibar Urban/West"
]

# División política de Tayikistán
divisiones_tayikistan = [
    "Dushanbe (City)", "Khatlon", "Sughd"
]

# División política de Tailandia
divisiones_tailandia = [
    "Amnat Charoen", "Ang Thong", "Bangkok", "Bueng Kan", "Buri Ram", "Chachoengsao", "Chai Nat", "Chaiyaphum", "Chanthaburi", "Chiang Mai", "Chiang Rai", "Chon Buri", "Chumphon", "Kalasin", "Kamphaeng Phet", "Kanchanaburi", "Khon Kaen", "Krabi", "Lampang", "Lamphun", "Loei", "Lopburi", "Mae Hong Son", "Maha Sarakham", "Mukdahan", "Nakhon Nayok", "Nakhon Pathom", "Nakhon Phanom", "Nakhon Ratchasima", "Nakhon Sawan", "Nakhon Si Thammarat", "Nan", "Narathiwat", "Nong Bua Lam Phu", "Nong Khai", "Nonthaburi", "Pathum Thani", "Pattani", "Phangnga", "Phatthalung", "Phayao", "Phetchabun", "Phetchaburi", "Phichit", "Phitsanulok", "Phra Nakhon Si Ayutthaya", "Phrae", "Phuket", "Prachin Buri", "Prachuap Khiri Khan", "Ranong", "Ratchaburi", "Rayong", "Roi Et", "Sa Kaeo", "Sakon Nakhon", "Samut Prakan", "Samut Sakhon", "Samut Songkhram", "Saraburi", "Satun", "Sing Buri", "Sisaket", "Songkhla", "Sukhothai", "Suphan Buri", "Surat Thani", "Surin", "Tak", "Trang", "Trat", "Ubon Ratchathani", "Udon Thani", "Uthai Thani", "Uttaradit", "Yala", "Yasothon"
]

# División política de Timor Oriental
divisiones_timor_oriental = [
    "Aileu", "Ainaro", "Baucau", "Bobonaro", "Cova Lima", "Dili", "Ermera", "Lautem", "Liquiçá", "Manatuto", "Manufahi", "Oecusse", "Viqueque"
]

# División política de Togo
divisiones_togo = [
    "Centrale", "Kara", "Maritime", "Plateaux", "Savanes"
]

# División política de Tonga
divisiones_tonga = [
    "Eua", "Ha'apai", "Niuas", "Tongatapu", "Vava'u"
]

# División política de Trinidad y Tobago
divisiones_trinidad_y_tobago = [
    "Arima", "Chaguanas", "Couva-Tabaquite-Talparo", "Diego Martin", "Eastern Tobago", "Penal-Debe", "Point Fortin", "Port of Spain", "Princes Town", "San Fernando", "San Juan-Laventille", "Sangre Grande", "Siparia", "Tunapuna-Piarco", "Western Tobago"
]

# División política de Túnez
divisiones_tunez = [
    "Ariana", "Beja", "Ben Arous", "Bizerte", "Gabes", "Gafsa", "Jendouba", "Kairouan", "Kasserine", "Kebili", "Kef", "Mahdia", "Manouba", "Medenine", "Monastir", "Nabeul", "Sfax", "Sidi Bouzid", "Siliana", "Sousse", "Tataouine", "Tozeur", "Tunis", "Zaghouan"
]

# División política de Turquía
divisiones_turquia = [
    "Adana", "Adiyaman", "Afyonkarahisar", "Agri", "Aksaray", "Amasya", "Ankara", "Antalya", "Ardahan", "Artvin", "Aydin", "Balikesir", "Bartin", "Batman", "Bayburt", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankiri", "Çorum", "Denizli", "Diyarbakir", "Düzce", "Edirne", "Elazig", "Erzincan", "Erzurum", "Eskisehir", "Gaziantep", "Giresun", "Gümüshane", "Hakkari", "Hatay", "Igdir", "Isparta", "Istanbul", "Izmir", "Kahramanmaras", "Karabük", "Karaman", "Kars", "Kastamonu", "Kayseri", "Kilis", "Kirikkale", "Kirklareli", "Kirsehir", "Kocaeli", "Konya", "Kütahya", "Malatya", "Manisa", "Mardin", "Mersin", "Mugla", "Mus", "Nevsehir", "Nigde", "Ordu", "Osmaniye", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Sanliurfa", "Sirnak", "Tekirdag", "Tokat", "Trabzon", "Tunceli", "Usak", "Van", "Yalova", "Yozgat", "Zonguldak"
]

# División política de Turkmenistán
divisiones_turkmenistan = [
    "Ahal", "Balkan", "Dashoguz", "Lebap", "Mary"
]

# División política de Tuvalu
divisiones_tuvalu = [
    "Funafuti", "Nanumanga", "Nanumea", "Niutao", "Nui", "Nukufetau", "Nukulaelae", "Vaitupu"
]

# División política de Uganda
divisiones_uganda = [
    "Central Region", "Eastern Region", "Northern Region", "Western Region"
]

# División política de Ucrania
divisiones_ucrania = [
    "Cherkasy", "Chernihiv", "Chernivtsi", "Dnipropetrovsk", "Donetsk", "Ivano-Frankivsk", "Kharkiv", "Kherson", "Khmelnytskyi", "Kiev", "Kiev City", "Kirovohrad", "Luhansk", "Lviv", "Mykolaiv", "Odessa", "Poltava", "Rivne", "Sevastopol City", "Sumy", "Ternopil", "Vinnytsia", "Volyn", "Zakarpattia", "Zaporizhia", "Zhytomyr"
]

# División política de Emiratos Árabes Unidos
divisiones_emiratos_arabes_unidos = [
    "Abu Dhabi", "Ajman", "Dubai", "Fujairah", "Ras Al Khaimah", "Sharjah", "Umm Al Quwain"
]

# División política de Reino Unido
divisiones_reino_unido = [
    "England", "Northern Ireland", "Scotland", "Wales"
]

# División política de Estados Unidos
divisiones_estados_unidos = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

# División política de Uruguay
divisiones_uruguay = [
    "Artigas", "Canelones", "Cerro Largo", "Colonia", "Durazno", "Flores", "Florida", "Lavalleja", "Maldonado", "Montevideo", "Paysandú", "Río Negro", "Rivera", "Rocha", "Salto", "San José", "Soriano", "Tacuarembó", "Treinta y Tres"
]

# División política de Uzbekistán
divisiones_uzbekistan = [
    "Andijan", "Bukhara", "Fergana", "Jizzakh", "Karakalpakstan", "Namangan", "Navoiy", "Qashqadaryo", "Samarqand", "Sirdaryo", "Surxondaryo", "Tashkent", "Xorazm"
]

# División política de Vanuatu
divisiones_vanuatu = [
    "Malampa", "Penama", "Sanma", "Shefa", "Tafea", "Torba"
]

# División política de Ciudad del Vaticano
divisiones_ciudad_del_vaticano = [
    "Vatican City"
]

# División política de Venezuela
divisiones_venezuela = [
    "Amazonas", "Anzoátegui", "Apure", "Aragua", "Barinas", "Bolívar", "Carabobo", "Cojedes", "Delta Amacuro", "Falcón", "Guárico", "Lara", "Mérida", "Miranda", "Monagas", "Nueva Esparta", "Portuguesa", "Sucre", "Táchira", "Trujillo", "Vargas", "Yaracuy", "Zulia"
]

# División política de Vietnam
divisiones_vietnam = [
    "An Giang", "Bà Rịa-Vũng Tàu", "Bắc Giang", "Bắc Kạn", "Bạc Liêu", "Bắc Ninh", "Bến Tre", "Bình Định", "Bình Dương", "Bình Phước", "Bình Thuận", "Cà Mau", "Cao Bằng", "Đắk Lắk", "Đắk Nông", "Điện Biên", "Đồng Nai", "Đồng Tháp", "Gia Lai", "Hà Giang", "Hà Nam", "Hà Tây", "Hà Tĩnh", "Hải Dương", "Hải Phòng", "Hòa Bình", "Hồ Chí Minh (city)", "Hưng Yên", "Khánh Hòa", "Kiên Giang", "Kon Tum", "Lai Châu", "Lâm Đồng", "Lạng Sơn", "Lào Cai", "Long An", "Nam Định", "Nghệ An", "Ninh Bình", "Ninh Thuận", "Phú Thọ", "Phú Yên", "Quảng Bình", "Quảng Nam", "Quảng Ngãi", "Quảng Ninh", "Quảng Trị", "Sóc Trăng", "Sơn La", "Tây Ninh", "Thái Bình", "Thái Nguyên", "Thanh Hóa", "Thừa Thiên-Huế", "Tiền Giang", "Trà Vinh", "Tuyên Quang", "Vĩnh Long", "Vĩnh Phúc", "Yên Bái"
]

# División política de Yemen
divisiones_yemen = [
    "'Adan", "Abyan", "Al Bayda", "Al Hudaydah", "Al Jawf", "Al Mahrah", "Al Mahwit", "Amanat Al Asimah (Sana'a City)", "Dhamar", "Hadhramaut", "Hajjah", "Ibb", "Lahij", "Ma'rib", "Raymah", "Sa'dah", "Shabwah", "Ta'izz"
]

# División política de Zambia
divisiones_zambia = [
    "Central", "Copperbelt", "Eastern", "Luapula", "Lusaka", "Muchinga", "Northern", "North-Western", "Southern", "Western"
]

# División política de Zimbabue
divisiones_zimbabue = [
    "Bulawayo", "Harare", "Manicaland", "Mashonaland Central", "Mashonaland East", "Mashonaland West", "Masvingo", "Matabeleland North", "Matabeleland South", "Midlands"
]



In [32]:
# Reemplazar los valores nulos por un valor vacío
aa['bajas_tripulacion'] = aa['bajas_tripulacion'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['bajas_tripulacion'] = aa['bajas_tripulacion'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['bajas_tripulacion'] = pd.to_numeric(aa['bajas_tripulacion'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['bajas_tripulacion'] = aa['bajas_tripulacion'].fillna(pd.NA).astype('Int64')


/tmp/ipykernel_39968/3341154712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tripulacion'] = aa['bajas_tripulacion'].fillna('')
/tmp/ipykernel_39968/3341154712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tripulacion'] = aa['bajas_tripulacion'].str.replace('?', '')
/tmp/ipykernel_39968/3341154712.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [33]:
# Reemplazar los valores nulos por un valor vacío
aa['bajas_tierra'] = aa['bajas_tierra'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['bajas_tierra'] = aa['bajas_tierra'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['bajas_tierra'] = pd.to_numeric(aa['bajas_tierra'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['bajas_tierra'] = aa['bajas_tierra'].fillna(pd.NA).astype('Int64')

/tmp/ipykernel_39968/1730188941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tierra'] = aa['bajas_tierra'].fillna('')
/tmp/ipykernel_39968/1730188941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['bajas_tierra'] = aa['bajas_tierra'].str.replace('?', '')
/tmp/ipykernel_39968/1730188941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [34]:
# Reemplazar los valores nulos por un valor vacío
aa['pasajeros'] = aa['pasajeros'].fillna('')

# Reemplazar los valores de ? por un valor vacío
aa['pasajeros'] = aa['pasajeros'].str.replace('?', '')

# Utilizar pd.to_numeric con errors='coerce' para convertir a números y mantener NaN en lugar de eliminar filas
aa['pasajeros'] = pd.to_numeric(aa['pasajeros'], errors='coerce')

# Convertir la columna 'tripulantes' a tipo entero (int) manteniendo NaN
aa['pasajeros'] = aa['pasajeros'].fillna(pd.NA).astype('Int64')

/tmp/ipykernel_39968/173493017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['pasajeros'] = aa['pasajeros'].fillna('')
/tmp/ipykernel_39968/173493017.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['pasajeros'] = aa['pasajeros'].str.replace('?', '')
/tmp/ipykernel_39968/173493017.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [35]:
def limpiar_descripcion(texto):
    # Utiliza una expresión regular para eliminar caracteres especiales, incluido '\r'
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', re.sub(r'[\r]', ' ', texto))
    return texto_limpio

# debido a '\r' tuve algunos inconvenientes.
aa['Descripcion'] = aa['Descripcion'].apply(limpiar_descripcion)
aa['operadora'] = aa['operadora'].apply(limpiar_descripcion)
aa['ruta'] = aa['ruta'].apply(limpiar_descripcion)



/tmp/ipykernel_39968/1399452123.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['Descripcion'] = aa['Descripcion'].apply(limpiar_descripcion)
/tmp/ipykernel_39968/1399452123.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['operadora'] = aa['operadora'].apply(limpiar_descripcion)
/tmp/ipykernel_39968/1399452123.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [36]:
# guardo en csv. 
aa.to_csv('avis.csv',index=False,sep='|',encoding='utf-8')

In [37]:
aa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4991 entries, 0 to 5007
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fecha              4991 non-null   datetime64[ns]
 1   hora               4991 non-null   object        
 2   pais               4991 non-null   object        
 3   operadora          4991 non-null   object        
 4   vuelo_numero       4991 non-null   object        
 5   ruta               4991 non-null   object        
 6   tipo_aeronave      4991 non-null   object        
 7   registro_aeronave  189 non-null    float64       
 8   abordo             4991 non-null   int64         
 9   pasajeros          4787 non-null   Int64         
 10  tripulantes        4789 non-null   Int64         
 11  bajas              4991 non-null   int64         
 12  bajas_pasajeros    4770 non-null   Int64         
 13  bajas_tripulacion  4771 non-null   Int64         
 14  bajas_tierra 

In [38]:
# Supongamos que tu DataFrame se llama 'df'
conteo_other = aa['pais'].value_counts().get('other', 0)

print(f"'other' se repite {conteo_other} veces en la columna 'pais'.")

'other' se repite 1295 veces en la columna 'pais'.
